Some curated sources:

 - [Socket Programming HOWTO - Python docs](https://docs.python.org/3/howto/sockets.html)
 - [Official docs for socket module](https://docs.python.org/3.10/library/socket.html)
 - [PyMOTW3 page for socket module](https://pymotw.com/3/socket/index.html#module-socket)
 - [Beej's Guide to Network Programming](https://beej.us/guide/bgnet/) - If you know C, this is an amazing resource. 

#### Socket

>A socket is one endpoint of a two-way communication link between two programs running on the network. A socket is bound to a port number so that the TCP layer can identify the application that data is destined to be sent to. [Source](https://docs.oracle.com/javase/tutorial/networking/sockets/definition.html)

Be careful to note that socket is not the connection. It is an endpoint of a connection. Also note that socket is an abstract entity. It doesn't have any physical manifestation. 

There are actually many types of sockets. The main ones are:

- Stream sockets (`SOCK_STREAM`) - TCP protocol is used. Reliable
- Datagram sockets (`SOCK_DGRAM`) - UDP protocol is used. Connectionless. Unreliable
- Raw  sockets (`SOCK_RAW`) - No protocol is used


We'll be mainly concerned with Stream sockets as our Web and many other Internet services use them exclusively. 

An endpoint is a combination of an IP address and a port number. Every TCP connection can be uniquely identified by its two endpoints. That way you can have multiple connections between your host and the server. 

Each computer (or some other device), when connected to Internet, is assigned an IP address. But your computer may be running many Internet based applications (such as web browser). So how does a data packet arrive at correct application? The answer is port. Each application has its own port number. So the combination of IP address and port number uniquely identifies the exact destination. The same thing goes for server too. Every server also has an IP address and a port where it listens to incoming requests from clients (such as applications running on your computer). This means a connection is established between a client (your computer) and a server (such as google.com). This connection is TCP based and each endpoint (that is socket) is uniquely identifiable by a combination of IP address and port. 

A word about IP addresses. You may have seen IP addresses like 129.168.11.135. There are basically two versions for assigning IP addresses. The first one, which is also older and prevalent, is **IPv4**. In this version, an IP address is defined as 32 bit number so we can come up with some 4.20 (2^32) billion addresses. In old days, 4 billion was a big number but it was later realised that there'd be far more devices in future and we'd run out of all the addresses of IPv4 could provide. So we came up with **IPv6** which defines IP address as a 128 bit number. However, IPv6 is still in the midst of adaptation. 

In socket programming, we usually have to specify which address family we want to use. For example, in Python, `AF_INET` means IPv4 address family and `AF_INET6` means IPv6 address family. There is one other address family, `AF_UNIX`, but we'll ignore that. 

**So, sockets have two primary properties controlling the way they send data: the address family controls the OSI network layer protocol used and the socket type controls the transport layer protocol.**

---

#### 10,000 Ft Overview of how sockets are created and used

`class socket.socket(family=AF_INET, type=SOCK_STREAM, proto=0, fileno=None)`

An instance of this class represents a socket. After creating this socket, you `bind()` it to a server address and then make it to `listen()`thereby enabling a server to accept connections. Finally, this server `accept()` incoming connection (this returns a new `conn` socket object and client `address`). Once connection is established, it `recv()` message from client via `conn` object. This server can also `send()`( or `sendall()`) messages to client using the `conn` object. 

On client side, we again create an instance of above class which represents client side socket. To connect with server, we use this socket to `connect(server_address)`. After that, we `sendall(message)` to server or `recv()` message from server.  

![](images/socketflow.png)


---

##### Example 1

[Source for following example](https://stackabuse.com/basic-socket-programming-in-python/)

In order to make use of the socket functionality, we use `socket` module. In the example code shown below the Python `time` module is imported as well in order to simuate the weather station and to simplyfy time calculations.

In this case, both the client and the server run on the same computer. A socket has a corresponding port number, which is 23456 in our case. If desired, you may choose a different port number from the unrestricted number range between 1024 and 65535.

#### The Server

Having loaded the additional Python `socket` module an Internet streaming socket is created using the `socket.socket` class with two parameters `socket.AF_INET` and `socket.SOCK_STREAM`. The retrieval of the hostname, the fully qualified domain name, and the IP address is done by the methods `gethostname()`, `getfqdn()`, and `gethostbyname()` respectively. 

Next, the socket is bound to the IP address and the port number 23456 with the help of the `bind()` method. 

With the help of the `listen()` method the server listens for incoming connections on the specified port. In the `while` loop the server waits for incoming requests and accepts them using the `accept()` method. The data submitted by the client is read via `recv()` method as chunks of 64 bytes, and simply output to stdout. Finally, the current connection is closed if no further data is sent from the client. 

In [65]:
%%file server.py

import socket

# create TCP/IP socket
sock = socket.socket(socket.AF_INET,socket.SOCK_STREAM)

#retrieve local hostname
local_hostname = socket.gethostname()

#get fully qualified domain name
local_fqdn =  socket.getfqdn()

#get the according IP address
ip_address = socket.gethostbyname(local_hostname)

#output hostname, domain name and IP address
print("working on %s (%s) with %s" % (local_hostname, local_fqdn, ip_address))

#bind the socket to the port 23456
server_address = (ip_address, 23456)
print('starting up on %s port %s' % server_address)
sock.bind(server_address)

#listen for incoming connection (server mode) with two connection at a time
sock.listen(2)

while True:
    #wait for a connection
    print("waiting for a connection")
    connection, client_address = sock.accept()
    try:
        #show who connected to us
        print('connection from', client_address)
        
        #receive the data in small chunks and print it
        while True:
            data = connection.recv(64)
            if data:
                #output received data
                print('Data: %s' %data)
            else:
                #no more data --quit the loop`
                print("no more data")
                break
    finally:
        connection.close()

Overwriting server.py


#### The Client

Now we will have a look at the client side. The Python code is mostly similar to the server side, except for the usage of the socket - the client uses the `connect()` method, instead. In a `for` loop the temperature data is sent to the server using the `sendall()` method. The call of the `time.sleep(2)` method pauses the client for two seconds before it sends another temperature reading. After all the temperature data is sent from the list the connection is finally closed using the `close()` method.

In [67]:
%%file client.py

import socket 
import time

#create TCP/IP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

#retrieve local hostname
local_hostname = socket.gethostname()
local_fqdn = socket.getfqdn()
ip_address = socket.gethostbyname(local_hostname)

server_address = (ip_address, 23456)
sock.connect(server_address)

print("connection to %s (%s) with %s" %(local_hostname, local_fqdn, ip_address))

#define example data to be sent to the server
temperature_data = ['15','33','44','12','11','26','22','34','40','18','21','29','42']

for entry in temperature_data:
    print("data: %s" %entry)
    new_data = str("temp: %s\n" %entry).encode("utf-8")
    sock.sendall(new_data)
    time.sleep(4)

    #close connection
sock.close()    

Overwriting client.py


To run both server and the client program, open two terminal windows and issue the following commands - one per terminal window and in the following order - 

    python server.py
    python client.py
    
Also note that don't run these scripts inside Notebook, it won't work. 


#### Sockets are blocking by default

[Source of following](https://realpython.com/python-sockets/#blocking-calls)

A socket function or method that temporarily suspends your application is a blocking call. For example, `.accept()`, `.connect()`, `.send()`, and `.recv()` block, meaning they don’t return immediately. Blocking calls have to wait on system calls (I/O) to complete before they can return a value. So you, the caller, are blocked until they’re done or a timeout or other error occurs.  Non-blocking mode is supported with `.setblocking()`.

A socket object can be in one of three modes: blocking, non-blocking, or timeout. Sockets are by default always created in blocking mode, but this can be changed by calling `setdefaulttimeout()`. Blocking socket calls can be set to non-blocking mode so they return immediately. If you do this, then you’ll need to at least refactor or redesign your application to handle the socket operation when it’s ready.

Because the call returns immediately, data may not be ready. The callee is waiting on the network and hasn’t had time to complete its work. If this is the case, then the current status is the `errno` value `socket.EWOULDBLOCK`. ([See this](https://stackoverflow.com/questions/3647539/socket-error-errno-ewouldblock) to know more about `socket.EWOULDBLOCK`)

From [Python docs: Notes on socket timeout](https://docs.python.org/3.10/library/socket.html#notes-on-socket-timeouts) -

- In _blocking mode_, operations block until complete or the system returns an error (such as connection timed out).

- _In non-blocking mode_, operations fail (with an error that is unfortunately system-dependent) if they cannot be completed immediately: functions from the `select` can be used to know when and whether a socket is available for reading or writing.

- _In timeout mode_, operations fail if they cannot be completed within the timeout specified for the socket (they raise a `timeout` exception) or if the system returns an error.


---

API Structure

 - Exceptions
  - `socket.error`
  - `socket.herror`
  - `socket.gaierror`
  - `socket.timeout`
 - Constants
  - `AF_*` address families such as `AF_INET`
  - `SOCK_*` socket type
  - `IPROTO_*` protocol constants
  - `MSG_*` 
  - many more
 - Class `socket.socket(family=AF_INET, type=SOCK_STREAM, proto=0, fileno=None)` - for creating socket objects
 - Function part 1 - these functions create sockets
  - `socketpair()` - returns 2-tuple. Each element represents a socket
  - `create_connection(address)` - returns and then connects this socket to server
  - `create_server()` - returns a server socket object
 - socket methods
  - `.accept()`
  - `.bind()`
  - `.close()`
  - `.connect()`
  - `.connect_ex()`
  - `.getpeername()`
  - `.getsockname()`
  - `.getsockopt()`
  - `.listen()`
  - `.recv()`
  - `.recvfrom()`
  - `.recvmsg()`
  - `.send()`
  - `.sendall()`
  - `.sendmsg()`
  - `.shutdown()`
  - `.setsockopt()`
  - `.setblocking()`
  - `.share()`, `.settimeout()`, `.sendfile()`
  - attributes `family`, `type`, `proto`
 - Functions part 2 
  - `has_dualstack_ipv6()` - returns `True` if platform supports IPv6 compatible socket creation
  - `gethostname()`
  - `gethostbyname()`
  - `gethostbyname_ex()`
  - `getfqdn()`
  - `gethostbyaddr()`
  - `getservbyname()`
  - `getservbyport()`
  - `getprotobyname()`
  - `getaddrinfo()`
  - `getnameinfo()`
  - `sethostname()`
  - `inet_aton()`
  - `inet_ntoa()`
  - `inet_pton()`
  - `inet_ntop()`

#### Module functions

[Source](https://pymotw.com/3/socket/)

#### `gethostname()`

To find the official name of the current host, use `gethostname()`

#### `gethostbyname()`

Use `gethostbyname()` to consult the OS hostname resolution API and convert the name of a server to its numerical address.

In [7]:
from socket import *

print(gethostname())


#


HOSTS = ['apu','pymotw.com','www.python.org','nosuchname']

for host in HOSTS:
    try:
        print('{} : {}'.format(host, gethostbyname(host)))
    except error as msg:
        print('{} : {}'.format(host, msg))

pc
apu : [Errno 11001] getaddrinfo failed
pymotw.com : 185.199.109.153
www.python.org : 199.232.20.223
nosuchname : [Errno 11001] getaddrinfo failed


#### `gethostbyname_ex()`

For access to more naming information about a server, use `gethostbyname_ex()`. It returns the canonical hostname of the server, any aliases, and all of the available IP addresses that can be used to reach it.

Having all known IP addresses for a server lets a client implement its own load balancing or fail-over algorithms.

In [8]:

HOSTS = ['apu','pymotw.com','www.python.org','nosuchname', 'google.com']

for host in HOSTS:
    print(host)
    try:
        name, aliases, addresses = gethostbyname_ex(host)
        print('  Hostname:', name)
        print('  Aliases :', aliases)
        print(' Addresses:', addresses)
    except error as msg:
        print('ERROR:', msg)
    print()

apu
ERROR: [Errno 11001] getaddrinfo failed

pymotw.com
  Hostname: pymotw.com
  Aliases : []
 Addresses: ['185.199.109.153', '185.199.108.153', '185.199.111.153', '185.199.110.153']

www.python.org
  Hostname: dualstack.python.map.fastly.net
  Aliases : ['www.python.org']
 Addresses: ['199.232.20.223']

nosuchname
ERROR: [Errno 11001] getaddrinfo failed

google.com
  Hostname: google.com
  Aliases : []
 Addresses: ['142.250.193.238']



#### `getfqdn()`

Use `getfqdn()` to convert a partial name to a fully qualified domain name.

The name returned will not necessarily match the input argument in any way if the input is an alias, such as `www` is here.

In [9]:
for host in ['apu', 'pymotw.com']:
    print('{:>10} : {}'.format(host, getfqdn(host)))

       apu : apu
pymotw.com : pymotw.com


#### `gethostbyaddr()`

When the address of a server is available, use `gethostbyaddr()` to do a “reverse” lookup for the name. The return value is a tuple containing the full hostname, any aliases, and all IP addresses associated with the name.

In [12]:
hostname, aliases, addresses = gethostbyaddr('142.250.193.238')

print('Hostname :', hostname)
print('Aliases  :', aliases)
print('Addresses:', addresses)

Hostname : del11s18-in-f14.1e100.net
Aliases  : []
Addresses: ['142.250.193.238']


#### Finding Service Information

####  `getservbyname()`

In addition to an IP address, each socket address includes an integer port number. Many applications can run on the same host, listening on a single IP address, but only one socket at a time can use a port at that address. The combination of IP address, protocol, and port number uniquely identify a communication channel and ensure that messages sent through a socket arrive at the correct destination.

Some of the port numbers are pre-allocated for a specific protocol. For example, communication between email servers using SMTP occurs over port number 25 using TCP, and web clients and servers use port 80 for HTTP. The port numbers for network services with standardized names can be looked up with `getservbyname()`

Although a standardized service is unlikely to change ports, looking up the value with a system call instead of hard-coding it is more flexible when new services are added in the future.

In [13]:
from urllib.parse import urlparse

URLS = [
    'http://www.python.org',
    'https://www.mybank.com',
    'ftp://prep.ai.mit.edu',
    'gopher://gopher.micro.umn.edu',
    'smtp://mail.example.com',
    'imap://mail.example.com',
    'imaps://mail.example.com',
    'pop3://pop.example.com',
    'pop3s://pop.example.com',
]

for url in URLS:
    parsed_url = urlparse(url)
    port = getservbyname(parsed_url.scheme)
    print('{:>6} : {}'.format(parsed_url.scheme, port))

  http : 80
 https : 443
   ftp : 21
gopher : 70
  smtp : 25
  imap : 143
 imaps : 993
  pop3 : 110
 pop3s : 995


#### `getservbyport(port[, protocolname])`

To reverse the service port lookup, use `getservbyport()`. The reverse lookup is useful for constructing URLs to services from arbitrary addresses. The optional protocol name, if given, should be `tcp` or `udp`, otherwise any protocol will match.

In [16]:
for port in [80, 443, 21, 70, 25, 143, 993, 110, 995]:
    url = '{}://example.com/'.format(getservbyport(port))
    print(url)

http://example.com/
https://example.com/
ftp://example.com/
gopher://example.com/
smtp://example.com/
imap://example.com/
imaps://example.com/
pop3://example.com/
pop3s://example.com/


In [19]:
getservbyport(443, 'tcp') #, getservbyport(55443, 'tcp') ->error

'https'

#### `getprotobyname(protocolname)`

The number assigned to a transport protocol can be retrieved with `getprotobyname()`. This translates an internet protocol name (for example, `icmp`) to a constant suitable for passing as the (optional) third argument to the `socket()` function. This is usually only needed for sockets opened in “raw” mode (`SOCK_RAW`); for the normal socket modes, the correct protocol is chosen automatically if the protocol is omitted or zero.

The values for protocol numbers are standardized, and defined as constants in `socket` with the prefix `IPPROTO_`.

In [25]:
import socket

def get_constants(prefix):
    """Create a dictionary mapping socket module
    constants to their names.
    """
    return {
        getattr(socket, n): n
        for n in dir(socket)
        if n.startswith(prefix)}


protocols = get_constants('IPPROTO_')
print(protocols,'\n')

for name in ['icmp', 'udp', 'tcp']:
    proto_num = socket.getprotobyname(name)
    const_name = protocols[proto_num]
    print('{:>4} -> {:2d} (socket.{:<12} = {:2d})'.format(
        name, proto_num, const_name,
        getattr(socket, const_name)))

{51: 'IPPROTO_AH', 7: 'IPPROTO_CBT', 60: 'IPPROTO_DSTOPTS', 8: 'IPPROTO_EGP', 50: 'IPPROTO_ESP', 44: 'IPPROTO_FRAGMENT', 3: 'IPPROTO_GGP', 0: 'IPPROTO_IP', 78: 'IPPROTO_ICLFXBM', 1: 'IPPROTO_ICMP', 58: 'IPPROTO_ICMPV6', 22: 'IPPROTO_IDP', 2: 'IPPROTO_IGMP', 9: 'IPPROTO_IGP', 4: 'IPPROTO_IPV4', 41: 'IPPROTO_IPV6', 115: 'IPPROTO_L2TP', 256: 'IPPROTO_MAX', 77: 'IPPROTO_ND', 59: 'IPPROTO_NONE', 113: 'IPPROTO_PGM', 103: 'IPPROTO_PIM', 12: 'IPPROTO_PUP', 255: 'IPPROTO_RAW', 27: 'IPPROTO_RDP', 43: 'IPPROTO_ROUTING', 132: 'IPPROTO_SCTP', 5: 'IPPROTO_ST', 6: 'IPPROTO_TCP', 17: 'IPPROTO_UDP'} 

icmp ->  1 (socket.IPPROTO_ICMP =  1)
 udp -> 17 (socket.IPPROTO_UDP  = 17)
 tcp ->  6 (socket.IPPROTO_TCP  =  6)


#### Looking Up Server Addresses

#### `getaddrinfo(host, port, family=0, type=0, proto=0, flags=0)`

Translate the host/port argument into a sequence of 5-tuples that contain all the necessary arguments for creating a socket connected to that service. host is a domain name, a string representation of an IPv4/v6 address or `None`. port is a string service name such as `http`, a numeric port number or `None`.

`getaddrinfo()` converts the basic address of a service into a list of tuples with all of the information necessary to make a connection. The contents of each tuple will vary, containing different network families or protocols.

In [26]:
import socket


def get_constants(prefix):
    """Create a dictionary mapping socket module
    constants to their names.
    """
    return {
        getattr(socket, n): n
        for n in dir(socket)
        if n.startswith(prefix)
    }


families = get_constants('AF_')
types = get_constants('SOCK_')
protocols = get_constants('IPPROTO_')

for response in socket.getaddrinfo('www.python.org', 'http'):

    # Unpack the response tuple
    family, socktype, proto, canonname, sockaddr = response

    print('Family        :', families[family])
    print('Type          :', types[socktype])
    print('Protocol      :', protocols[proto])
    print('Canonical name:', canonname)
    print('Socket address:', sockaddr)
    print()

Family        : AF_INET6
Type          : SOCK_STREAM
Protocol      : IPPROTO_IP
Canonical name: 
Socket address: ('2a04:4e42:42::223', 80, 0, 0)

Family        : AF_INET
Type          : SOCK_STREAM
Protocol      : IPPROTO_IP
Canonical name: 
Socket address: ('199.232.20.223', 80)



See following example also. 

In [27]:
import socket


def get_constants(prefix):
    """Create a dictionary mapping socket module
    constants to their names.
    """
    return {
        getattr(socket, n): n
        for n in dir(socket)
        if n.startswith(prefix)
    }


families = get_constants('AF_')
types = get_constants('SOCK_')
protocols = get_constants('IPPROTO_')

responses = socket.getaddrinfo(
    host='www.python.org',
    port='http',
    family=socket.AF_INET,
    type=socket.SOCK_STREAM,
    proto=socket.IPPROTO_TCP,
    flags=socket.AI_CANONNAME,
)

for response in responses:
    # Unpack the response tuple
    family, socktype, proto, canonname, sockaddr = response

    print('Family        :', families[family])
    print('Type          :', types[socktype])
    print('Protocol      :', protocols[proto])
    print('Canonical name:', canonname)
    print('Socket address:', sockaddr)
    print()

Family        : AF_INET
Type          : SOCK_STREAM
Protocol      : IPPROTO_TCP
Canonical name: dualstack.python.map.fastly.net
Socket address: ('199.232.20.223', 80)



#### IP Address Representations

Network programs written in C use the data type struct sockaddr to represent IP addresses as binary values (instead of the string addresses usually found in Python programs). To convert `IPv4` addresses between the Python representation and the C representation, use `inet_aton()` and `inet_ntoa()`.

In [28]:
import binascii
import socket
import struct
import sys

for string_address in ['192.168.1.1', '127.0.0.1']:
    packed = socket.inet_aton(string_address)
    print('Original:', string_address)
    print('Packed  :', binascii.hexlify(packed))
    print('Unpacked:', socket.inet_ntoa(packed))
    print()

Original: 192.168.1.1
Packed  : b'c0a80101'
Unpacked: 192.168.1.1

Original: 127.0.0.1
Packed  : b'7f000001'
Unpacked: 127.0.0.1



The related functions `inet_pton()` and `inet_ntop()` work with both IPv4 and IPv6 addresses, producing the appropriate format based on the address family parameter passed in.

In [29]:
import binascii
import socket
import struct
import sys

string_address = '2002:ac10:10a:1234:21e:52ff:fe74:40e'
packed = socket.inet_pton(socket.AF_INET6, string_address)

print('Original:', string_address)
print('Packed  :', binascii.hexlify(packed))
print('Unpacked:', socket.inet_ntop(socket.AF_INET6, packed))

Original: 2002:ac10:10a:1234:21e:52ff:fe74:40e
Packed  : b'2002ac10010a1234021e52fffe74040e'
Unpacked: 2002:ac10:10a:1234:21e:52ff:fe74:40e


Some functions have been left out.

##### Module Exploration

This sample program, based on the one in the standard library documentation, receives incoming messages and echos them back to the sender. It starts by creating a TCP/IP socket, then `bind()` is used to associate the socket with the server address. In this case, the address is `localhost`, referring to the current server, and the port number is 10000.

In [2]:
%%file echo_server.py

import socket
import sys

# Create a TCP/IP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Bind the socket to the port
server_address = ('localhost', 10000)
print('starting up on {} port {}'.format(*server_address))
sock.bind(server_address)

# Listen for incoming connections
sock.listen(1)

while True:
    # Wait for a connection
    print('waiting for a connection')
    connection, client_address = sock.accept()
    try:
        print('connection from', client_address)

        # Receive the data in small chunks and retransmit it
        while True:
            data = connection.recv(16)
            print('received {!r}'.format(data))
            if data:
                print('sending data back to the client')
                connection.sendall(data)
            else:
                print('no data from', client_address)
                break

    finally:
        # Clean up the connection
        connection.close()

Writing echo_server.py


In [3]:
%%file echo_client.py

import socket
import sys

# Create a TCP/IP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Connect the socket to the port where the server is listening
server_address = ('localhost', 10000)
print('connecting to {} port {}'.format(*server_address))
sock.connect(server_address)

try:

    # Send data
    message = b'This is the message.  It will be repeated.'
    print('sending {!r}'.format(message))
    sock.sendall(message)

    # Look for the response
    amount_received = 0
    amount_expected = len(message)

    while amount_received < amount_expected:
        data = sock.recv(16)
        amount_received += len(data)
        print('received {!r}'.format(data))

finally:
    print('closing socket')
    sock.close()

Writing echo_client.py


#### Easy Client Connections

#### `create_connection()`

Connect to a TCP service listening on the internet *address* (a 2-tuple (`host`, `port`)), and return the socket object. This is a higher-level function than `socket.connect()`: if host is a non-numeric hostname, it will try to resolve it for both `AF_INET` and `AF_INET6`, and then try to connect to all possible addresses in turn until a connection succeeds. This makes it easy to write clients that are compatible to both `IPv4` and `IPv6`.

TCP/IP clients can save a few steps by using the convenience function `create_connection()` to connect to a server. The function takes one argument, a two-value tuple containing the address of the server, and derives the best address to use for the connection

In [38]:
import socket
{
        getattr(socket, n): n
        for n in dir(socket)
        if n.startswith('AF_')
    }

{<AddressFamily.AF_APPLETALK: 16>: 'AF_APPLETALK',
 <AddressFamily.AF_BLUETOOTH: 32>: 'AF_BLUETOOTH',
 12: 'AF_DECnet',
 <AddressFamily.AF_INET: 2>: 'AF_INET',
 <AddressFamily.AF_INET6: 23>: 'AF_INET6',
 <AddressFamily.AF_IPX: 6>: 'AF_IPX',
 <AddressFamily.AF_IRDA: 26>: 'AF_IRDA',
 <AddressFamily.AF_LINK: 33>: 'AF_LINK',
 <AddressFamily.AF_SNA: 11>: 'AF_SNA',
 <AddressFamily.AF_UNSPEC: 0>: 'AF_UNSPEC'}

In [30]:
%%file socket1.py

import socket
import sys


def get_constants(prefix):
    """Create a dictionary mapping socket module
    constants to their names.
    """
    return {
        getattr(socket, n): n
        for n in dir(socket)
        if n.startswith(prefix)
    }


families = get_constants('AF_')
types = get_constants('SOCK_')
protocols = get_constants('IPPROTO_')

# Create a TCP/IP socket
#note that we didn't use `sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)`
#create_connection returns socket object just like socket.socket(...)
#also note the lack of socket.connect(server) expression

sock = socket.create_connection(('localhost', 10000))

print('Family  :', families[sock.family])
print('Type    :', types[sock.type])
print('Protocol:', protocols[sock.proto])
print()

try:

    # Send data
    message = b'This is the message.  It will be repeated.'
    print('sending {!r}'.format(message))
    sock.sendall(message)

    amount_received = 0
    amount_expected = len(message)

    while amount_received < amount_expected:
        data = sock.recv(16)
        amount_received += len(data)
        print('received {!r}'.format(data))

finally:
    print('closing socket')
    sock.close()

Writing socket1.py


`create_connection()` uses `getaddrinfo()` to find candidate connection parameters, and returns a socket opened with the first configuration that creates a successful connection. The family, type, and proto attributes can be examined to determine the type of socket being returned.

Output:

```
Family  : AF_INET
Type    : SOCK_STREAM
Protocol: IPPROTO_IP

sending b'This is the message.  It will be repeated.'
received b'This is the mess'
received b'age.  It will be'
received b' repeated.'
closing socket
```

#### Choosing an Address for Listening

It is important to bind a server to the correct address, so that clients can communicate with it. The previous examples all used `localhost` as the IP address, which limits connections to clients running on the same server. Use a public address of the server, such as the value returned by `gethostname()`, to allow other hosts to connect. This example modifies the echo server to listen on an address specified via a command line argument.

In [31]:
%%file echo_server1.py

import socket
import sys

# Create a TCP/IP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Bind the socket to the address given on the command line
server_name = sys.argv[1]
server_address = (server_name, 10000)
print('starting up on {} port {}'.format(*server_address))
sock.bind(server_address)
sock.listen(1)

while True:
    print('waiting for a connection')
    connection, client_address = sock.accept()
    try:
        print('client connected:', client_address)
        while True:
            data = connection.recv(16)
            print('received {!r}'.format(data))
            if data:
                connection.sendall(data)
            else:
                break
    finally:
        connection.close()

Writing echo_server1.py


A similar modification to the client program is needed before the server can be tested.

In [32]:
%%file echo_client1.py

import socket
import sys

# Create a TCP/IP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Connect the socket to the port on the server
# given by the caller
server_address = (sys.argv[1], 10000)
print('connecting to {} port {}'.format(*server_address))
sock.connect(server_address)

try:

    message = b'This is the message.  It will be repeated.'
    print('sending {!r}'.format(message))
    sock.sendall(message)

    amount_received = 0
    amount_expected = len(message)
    while amount_received < amount_expected:
        data = sock.recv(16)
        amount_received += len(data)
        print('received {!r}'.format(data))

finally:
    sock.close()

Writing echo_client1.py


**Note:-** The source material provided `hubert.hellfly.net` as argument which on my system raised `OSError`. So I provided `pc` as argument. This way, program ran correctly. 

Many servers have more than one network interface, and therefore more than one IP address. Rather than running separate copies of a service bound to each IP address, use the special address `INADDR_ANY` to listen on all addresses at the same time. Although socket defines a constant for `INADDR_ANY`, it is an integer value and must be converted to a dotted-notation string address before it can be passed to `bind()`. As a shortcut, use `0.0.0.0` or an empty string (`''`) instead of doing the conversion.

As a contrived example, let's say a server is using 4 IP addresses (say `111.111.111.1`, `111.111.111.2`, `111.111.111.3` and `111.111.111.4`) to listen to incoming connections. Instead of creating 4 sockets for each of these addresses, we simply `bind()` the server to `0.0.0.0` (or `''`). This way, server is able to listen on all of above 4 addresses. However, client still needs to specify one of these addresses to connect with server. 

In [33]:
%%file echo_server2.py

import socket
import sys

# Create a TCP/IP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Bind the socket to the address given on the command line
server_address = ('', 10000)
sock.bind(server_address)
print('starting up on {} port {}'.format(*sock.getsockname()))
sock.listen(1)

while True:
    print('waiting for a connection')
    connection, client_address = sock.accept()
    try:
        print('client connected:', client_address)
        while True:
            data = connection.recv(16)
            print('received {!r}'.format(data))
            if data:
                connection.sendall(data)
            else:
                break
    finally:
        connection.close()

Writing echo_server2.py


**Aside**

`127.0.0.1` is the `loopback address` (also known as `localhost`) and `0.0.0.0` is the IP address that is a *non-routable meta-address* used to indicate an invalid, unknown, or non-applicable destination.

`0.0.0.0` and `127.0.0.1` are easy to confuse, but let's keep remember that an address with four zeros has a few defined uses, whereas `127.0.0.1` instead has the very specific purpose of allowing a device to send messages to itself.

In earlier examples, we used `localhost` (that is, `127.0.0.1`) as server address. This means, your server will be accessible to the clients on the same machine. Any other machine on the network will not be able to access that server. On the other hand, `0.0.0.0` means your server is visible to outside world and any machine can connect to this server using appropriate server address. Also note that we haven't used the port number because, by default, all webservers use port `80` to listen on. If your server is using any other port, client has to specify port number alongwith IP address. Be clear that client can't use `0.0.0.0` address to connect with server. 

In essence, `0.0.0.0` means _"anywhere and everywhere"_, while `127.0.0.1` means _"precisely here and nowhere else"_.

See [this](https://stackoverflow.com/questions/20778771/what-is-the-difference-between-0-0-0-0-127-0-0-1-and-localhost) and [this](https://stackoverflow.com/questions/74233128/difference-between-0-0-0-0-and-127-0-0-1-on-windows-and-linux)

#### About `sock.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)`

In the notebook 'Python Concurrency - Part 1', there is an example of server code:

```
address = ('', 25000)
sock = socket(AF_INET, SOCK_STREAM) #(1st param: address family, 2nd param: socket type)
sock.setsockopt(SOL_SOCKET, SO_REUSEADDR,1)  # line 2
sock.bind(address)
sock.listen(5) 
while True:
    client,addr = sock.accept()
    ....
```

In line 2, we used the method `setsockopt(level, optname, value)`. I didn't quite understand this and [Python docs](https://docs.python.org/3.10/library/socket.html#socket.socket.setsockopt) wasn't of much help. This [SO Post](https://stackoverflow.com/questions/14388706/how-do-so-reuseaddr-and-so-reuseport-differ/14388707?utm_source=pocket_saves) touches on this.

In [59]:


sock1 = socket.socket(family=AF_INET, type=SOCK_STREAM, proto=0, fileno=None)
sock2 = socketpair()
sock2

(<socket.socket fd=1532, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 49334), raddr=('127.0.0.1', 49335)>,
 <socket.socket fd=1568, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 49335), raddr=('127.0.0.1', 49334)>)

In [60]:
socket.has_dualstack_ipv6()

True

#### Multi-connection server and client

[Source](https://realpython.com/python-sockets/#reference)

This section uses [selectors](https://docs.python.org/3/library/selectors.html) module which I am not even slightly familiar with. Although I can recall that in `asyncio`, the `select()` function is used to select event.

So, let us first try to explore `selectors` module to some extent. 

From [PyMOTW](https://pymotw.com/3/selectors/index.html#module-selectors):

The APIs in selectors are event-based, similar to `poll()` from `select`. There are several implementations and the module automatically sets the alias DefaultSelector to refer to the most efficient one for the current system configuration.

A selector object provides methods for specifying what events to look for on a socket, and then lets the caller wait for events in a platform-independent way. Registering interest in an event creates a `SelectorKey`, which holds the socket, information about the events of interest, and optional application data. The owner of the selector calls its `select()` method to learn about events. The return value is a sequence of key objects and a bitmask indicating what events have occurred. A program using a selector should repeatedly call `select()`, then handle the events appropriately.

The module is fairly small. At its core, it defines a class hiearchy:
```
BaseSelector
+-- SelectSelector
+-- PollSelector
+-- EpollSelector
+-- DevpollSelector
+-- KqueueSelector
```

However, we should use `DefaultSelector` class which is an alias for the best implementation for current platform. There are few terms we'll encounter later whih we should be aware of:


**`fileobj`** - File object registered. In the context of socket programming, a `fileobj` is socket object (for ex, an instance of `socket` class). In Unix philosophy, everything (e.g. sokets, pipes, devices and so on) is a file. The same is not true for Windows platform. On Windows, only sockets can be `fileobj`.  


**`events`**: An _event_ is a bitwise mask indicating which I/O events should be waited for on a given file object. Occasionally in some codes, term `mask` is also used. It can be a combination of the modules constants below:

Constant|Meaning
----------|-----
`EVENT_READ`| Available for read
`EVENT_WRITE`| Available for write

**`fd`** Underlying file descriptor. This is essentially a small integer associated with an open file (socket object in the current context) ready for read/write operation. 

**`data`** Optional opaque data associated to this file object: for example, this could be used to store a per-client session ID.



An instance (say `sel`) of `DefaultSelector` class has following main methods available:

**Note** - Many of these methods return an instance of `SelectorKey` instance. A `SelectorKey` is a `namedtuple` used to associate a file object to its underlying file descriptor, selected event mask and attached data. `SelectorKey` looks like -

`SelectorKey(fileobj=<socket.socket fd=256, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 10000)>, fd=256, events=1, data=<function accept at 0x0000003D1690B5E0>)`

`sel = selectors.DefaultSelector()`

`sel.register(fileobj, events, data=None)`- This returns a new `SelectorKey` instance, or raises a `ValueError` in case of invalid event mask or file descriptor, or `KeyError` if the file object is already registered.


`sel.unregister(fileobj)` - Returns the associated `SelectorKey` instance, or raises a `KeyError` if `fileobj` is not registered. 

`sel.selct(timeout = None)` - This returns a list of `(key, events)` tuples, one for each ready file object. `key` is the `SelectorKey` instance corresponding to a ready file object. events is a bitmask of events ready on this file object.

`sel.close()` - Close the selector.


#### Echo Server

The echo server example below uses the application data in the `SelectorKey` to register a callback function to be invoked on the new event. The main loop gets the callback from the key and passes the socket and event mask to it. As the server starts, it registers the `accept()` function to be called for read events on the main server socket. Accepting the connection produces a new socket, which is then registered with the `read()` function as a callback for read events.

In [86]:
%%file selectserver.py

import selectors
import socket

mysel = selectors.DefaultSelector()
keep_running = True


def read(connection):
    "Callback for read events"
    global keep_running

    client_address = connection.getpeername()
    print('read({})'.format(client_address))
    data = connection.recv(1024)
    if data:
        # A readable client socket has data
        print('  received {!r}'.format(data))
        connection.sendall(data)
    else:
        # Interpret empty result as closed connection
        print('  closing')
        mysel.unregister(connection)
        connection.close()
        # Tell the main loop to stop
        keep_running = False


def accept(sock):
    "Callback for new connections"
    new_connection, addr = sock.accept()
    print('accept({})'.format(addr))
    new_connection.setblocking(False)
    mysel.register(new_connection, selectors.EVENT_READ, read)


server_address = ('localhost', 10000)
print('starting up on {} port {}'.format(*server_address))
server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.setblocking(False)
server.bind(server_address)
server.listen(5)

mysel.register(server, selectors.EVENT_READ, accept)

while keep_running:
    print('waiting for I/O')
    for key, mask in mysel.select(timeout=1):
        print("mask is: ", mask)
        callback = key.data
        callback(key.fileobj)

print('shutting down')
mysel.close()

Overwriting selectserver.py


#### Echo Client

The echo client example below processes all of the I/O events in the main loop, instead of using callbacks. It sets up the selector to report read events on the socket, and to report when the socket is ready to send data. Because it is looking at two types of events, the client must check which occurred by examining the mask value. After all of its outgoing data has been sent, it changes the selector configuration to only report when there is data to read.

In [82]:
%%file selectclient.py

import selectors
import socket

mysel = selectors.DefaultSelector()
keep_running = True
outgoing = [b'It will be repeated.', b'This is the message.  ',]
bytes_sent = 0
bytes_received = 0

# Connecting is a blocking operation, so call setblocking()
# after it returns.
server_address = ('localhost', 10000)
print('connecting to {} port {}'.format(*server_address))
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect(server_address)
sock.setblocking(False)

# Set up the selector to watch for when the socket is ready
# to send data as well as when there is data to read.
mysel.register(sock,selectors.EVENT_READ | selectors.EVENT_WRITE,)

while keep_running:
    print('waiting for I/O')
    for key, mask in mysel.select(timeout=1):
        connection = key.fileobj
        client_address = connection.getpeername()
        print('client({})'.format(client_address))

        if mask & selectors.EVENT_READ:
            print('  ready to read')
            data = connection.recv(1024)
            if data:
                # A readable client socket has data
                print('  received {!r}'.format(data))
                bytes_received += len(data)

            # Interpret empty result as closed connection,
            # and also close when we have received a copy
            # of all of the data sent.
            keep_running = not (
                data or
                (bytes_received and
                 (bytes_received == bytes_sent))
            )

        if mask & selectors.EVENT_WRITE:
            print('  ready to write')
            if not outgoing:
                # We are out of messages, so we no longer need to
                # write anything. Change our registration to let
                # us keep reading responses from the server.
                print('  switching to read-only')
                mysel.modify(sock, selectors.EVENT_READ)
            else:
                # Send the next message.
                next_msg = outgoing.pop()
                print('  sending {!r}'.format(next_msg))
                sock.sendall(next_msg)
                bytes_sent += len(next_msg)

print('shutting down')
mysel.unregister(connection)
connection.close()
mysel.close()

Writing selectclient.py


The client tracks the amount of data it has sent, and the amount it has received. When those values match and are non-zero, the client exits the processing loop and cleanly shuts down by removing the socket from the selector and closing both the socket and the selector.


Output on server:

```
starting up on localhost port 10000
waiting for I/O
waiting for I/O
accept(('127.0.0.1', 59850))
waiting for I/O
read(('127.0.0.1', 59850))
  received b'This is the message.  It will be repeated.'
waiting for I/O
read(('127.0.0.1', 59850))
  closing
shutting down
```

Output on client:

```
connecting to localhost port 10000
waiting for I/O
client(('127.0.0.1', 10000))
  ready to write
  sending b'This is the message.  '
waiting for I/O
client(('127.0.0.1', 10000))
  ready to write
  sending b'It will be repeated.'
waiting for I/O
client(('127.0.0.1', 10000))
  ready to write
  switching to read-only
waiting for I/O
client(('127.0.0.1', 10000))
  ready to read
  received b'This is the message.  It will be repeated.'
shutting down
```


Alternatively, you can only run `selectserver.py` and use `telnet localhost 10000` command in 2 separate terminal windows. You can see server is able to connect (and respond) to both clients. 

[Source for following example](https://realpython.com/python-sockets/#reference)


In [92]:
%%file multiconnserver.py

import sys
import socket
import selectors
import types

sel = selectors.DefaultSelector()


def accept_wrapper(sock):
    conn, addr = sock.accept()  # Should be ready to read
    print(f"Accepted connection from {addr}")
    conn.setblocking(False)
    data = types.SimpleNamespace(addr=addr, inb=b"", outb=b"")
    events = selectors.EVENT_READ | selectors.EVENT_WRITE
    sel.register(conn, events, data=data)


def service_connection(key, mask):
    sock = key.fileobj
    data = key.data
    if mask & selectors.EVENT_READ:
        recv_data = sock.recv(1024)  # Should be ready to read
        if recv_data:
            data.outb += recv_data
        else:
            print(f"Closing connection to {data.addr}")
            sel.unregister(sock)
            sock.close()
    if mask & selectors.EVENT_WRITE:
        if data.outb:
            print(f"Echoing {data.outb!r} to {data.addr}")
            sent = sock.send(data.outb)  # Should be ready to write
            data.outb = data.outb[sent:]


if len(sys.argv) != 3:
    print(f"Usage: {sys.argv[0]} <host> <port>")
    sys.exit(1)

host, port = sys.argv[1], int(sys.argv[2])
lsock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
lsock.bind((host, port))
lsock.listen()
print(f"Listening on {(host, port)}")
lsock.setblocking(False)
sel.register(lsock, selectors.EVENT_READ, data=None)

try:
    while True:
        events = sel.select(timeout=None)
        for key, mask in events:
            if key.data is None:
                accept_wrapper(key.fileobj)
            else:
                service_connection(key, mask)
except KeyboardInterrupt:
    print("Caught keyboard interrupt, exiting")
finally:
    sel.close()

Overwriting multiconnserver.py


In [91]:
%%file multiconnclient.py

import sys
import socket
import selectors
import types

sel = selectors.DefaultSelector()
messages = [b"Message 1 from client.", b"Message 2 from client."]


def start_connections(host, port, num_conns):
    server_addr = (host, port)
    for i in range(0, num_conns):
        connid = i + 1
        print(f"Starting connection {connid} to {server_addr}")
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.setblocking(False)
        sock.connect_ex(server_addr)
        events = selectors.EVENT_READ | selectors.EVENT_WRITE
        data = types.SimpleNamespace(
            connid=connid,
            msg_total=sum(len(m) for m in messages),
            recv_total=0,
            messages=messages.copy(),
            outb=b"",
        )
        sel.register(sock, events, data=data)


def service_connection(key, mask):
    sock = key.fileobj
    data = key.data
    if mask & selectors.EVENT_READ:
        recv_data = sock.recv(1024)  # Should be ready to read
        if recv_data:
            print(f"Received {recv_data!r} from connection {data.connid}")
            data.recv_total += len(recv_data)
        if not recv_data or data.recv_total == data.msg_total:
            print(f"Closing connection {data.connid}")
            sel.unregister(sock)
            sock.close()
    if mask & selectors.EVENT_WRITE:
        if not data.outb and data.messages:
            data.outb = data.messages.pop(0)
        if data.outb:
            print(f"Sending {data.outb!r} to connection {data.connid}")
            sent = sock.send(data.outb)  # Should be ready to write
            data.outb = data.outb[sent:]


if len(sys.argv) != 4:
    print(f"Usage: {sys.argv[0]} <host> <port> <num_connections>")
    sys.exit(1)

host, port, num_conns = sys.argv[1:4]
start_connections(host, int(port), int(num_conns))

try:
    while True:
        events = sel.select(timeout=1)
        if events:
            for key, mask in events:
                service_connection(key, mask)
        # Check for a socket being monitored to continue.
        if not sel.get_map():
            break
except KeyboardInterrupt:
    print("Caught keyboard interrupt, exiting")
finally:
    sel.close()



Overwriting multiconnclient.py


Following is just a test program written by me to experiment. 

In [78]:
%%file testserver.py

from socket import *

sock = socket(AF_INET, SOCK_STREAM)
sock.bind(('localhost',11000))
print(sock.getsockname())
sock.listen(2)

while True:
    conn, addr = sock.accept()
    print("connected to: ", addr)
    while True:
        msg = conn.recv(1024)
        print("received msg: ", msg)
        if msg:
            conn.sendall(msg)
            print('msg sent: ', msg)
        else:
            sock.close()
            break
            

Overwriting testserver.py


In [89]:
%%file testclient.py

from socket import *

sock = socket(AF_INET, SOCK_STREAM)
a = sock.connect(('localhost',11000))
print(a)
print(sock.getsockname())

while True:
    inp = input("message to be sent: - ")
    if not inp:
        break
    binp = inp.encode('ascii')
    sock.sendall(binp)
    print("message sent: ", binp)
    a = sock.recv(1024)
    print("message received: ", a)
    

sock.close()

Overwriting testclient.py
